# Yanoya Data

## Data

### Data Source Python

In [0]:
import json
import altair as alt
import numpy as np
import pandas as pd

yny_data_json = '''
{
    "top": {
        "samples": [
            {
                "name": "Battery (Lithium-Ion)",
                "energy_density": 0.3,
                "energy_density_unit": "kWh/kg",
                "efficiency": 90,
                "efficiency_unit": "%",
                "effective_energy_density": 27.0,
                "effective_energy_density_unit": "kWh/kg",
                "weight_needed_for_500kwh": 18.51851851851852,
                "weight_needed_for_500kwh_unit": "kg"
            },
            {
                "name": "Hydrogen (Fuel Cell)",
                "energy_density": 33.6,
                "energy_density_unit": "kWh/kg",
                "efficiency": 60,
                "efficiency_unit": "%",
                "effective_energy_density": 2016.0,
                "effective_energy_density_unit": "kWh/kg",
                "weight_needed_for_500kwh": 0.24801587301587302,
                "weight_needed_for_500kwh_unit": "kg"
            },
            {
                "name": "Diesel/Biodiesel (Diesel Engine)",
                "energy_density": 10,
                "energy_density_unit": "kWh/kg",
                "efficiency": 45,
                "efficiency_unit": "%",
                "effective_energy_density": 450,
                "effective_energy_density_unit": "kWh/kg",
                "weight_needed_for_500kwh": 1.1111111111111112,
                "weight_needed_for_500kwh_unit": "kg"
            },
            {
                "name": "Bio Methanol",
                "energy_density": 5.5,
                "energy_density_unit": "kWh/kg",
                "efficiency": 40,
                "efficiency_unit": "%",
                "effective_energy_density": 220.0,
                "effective_energy_density_unit": "kWh/kg",
                "weight_needed_for_500kwh": 2.272727272727273,
                "weight_needed_for_500kwh_unit": "kg"
            },
            {
                "name": "Bio Ethanol",
                "energy_density": 7.8,
                "energy_density_unit": "kWh/kg",
                "efficiency": 35,
                "efficiency_unit": "%",
                "effective_energy_density": 273.0,
                "effective_energy_density_unit": "kWh/kg",
                "weight_needed_for_500kwh": 1.8315018315018314,
                "weight_needed_for_500kwh_unit": "kg"
            },
            {
                "name": "Jet Fuel (Aviation Turbofan)",
                "energy_density": 11.9,
                "energy_density_unit": "kWh/kg",
                "efficiency": 45,
                "efficiency_unit": "%",
                "effective_energy_density": 535.5,
                "effective_energy_density_unit": "kWh/kg",
                "weight_needed_for_500kwh": 0.9337068160597572,
                "weight_needed_for_500kwh_unit": "kg"
            }
        ]
    }
}
'''

yny_data_py = json.loads(yny_data_json)
yny_item_name = list(yny_data_py)[0]
yny_data_without_root_py = yny_data_py[yny_item_name]
yny_data_py

### Helpers

In [0]:
def yny_get(data, path):
    path_parts = path.split("/")
    def yny_path_rec(data, path_parts):
        if not path_parts:
            return data
        if isinstance(data, dict):
            if path_parts[0] in data:
                return yny_path_rec(data[path_parts[0]], path_parts[1:])
        if isinstance(data, list):
            if len(data)==1:
                return yny_path_rec(data[0], path_parts[1:])
        return None
    return yny_path_rec(data, path_parts)

def yny_get_list(data, path):
    path_parts = path.split("/")
    def yny_path_rec(data, path_parts, ret):
        if not path_parts:
            if isinstance(data, list):
                ret.extend(data)
            else:
                ret.append(data)
            return
        if isinstance(data, dict):
            if path_parts[0] in data:
                return yny_path_rec(data[path_parts[0]], path_parts[1:], ret)
        if isinstance(data, list) and path_parts[0]=="*":
            for el in data:
                yny_path_rec(el, path_parts[1:], ret)
        return None
    ret = []
    yny_path_rec(data, path_parts, ret)
    return ret

### JSON Format

In [0]:
json.dumps(yny_data_py)

### YAML Format

In [0]:
import yaml
yny_data_yaml = yaml.dump(yny_data_py)
print(yny_data_yaml,end=chr(10))

### Pandas Dataframe

In [0]:
yny_data_df = pd.DataFrame({
  'name': yny_get_list(yny_data_without_root_py, 'samples/*/name'),
  'energy_density_unit': yny_get_list(yny_data_without_root_py, 'samples/*/energy_density_unit'),
  'efficiency_unit': yny_get_list(yny_data_without_root_py, 'samples/*/efficiency_unit'),
  'effective_energy_density_unit': yny_get_list(yny_data_without_root_py, 'samples/*/effective_energy_density_unit'),
  'weight_needed_for_500kwh_unit': yny_get_list(yny_data_without_root_py, 'samples/*/weight_needed_for_500kwh_unit'),
  'energy_density': yny_get_list(yny_data_without_root_py, 'samples/*/energy_density'),
  'efficiency': yny_get_list(yny_data_without_root_py, 'samples/*/efficiency'),
  'effective_energy_density': yny_get_list(yny_data_without_root_py, 'samples/*/effective_energy_density'),
  'weight_needed_for_500kwh': yny_get_list(yny_data_without_root_py, 'samples/*/weight_needed_for_500kwh')
  
})
yny_data_df

### CSV Format

In [0]:
print(yny_data_df.to_csv(index=False),end=chr(10))

### Line Chart

In [0]:
chart = alt.Chart(yny_data_df).mark_line().encode(x='energy_density',y='efficiency')
chart

### Pie Chart

In [0]:
chart = alt.Chart(yny_data_df).mark_arc().encode(
 theta="energy_density",
 color="efficiency"
)
chart